In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더한다."""
    
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱한다."""
    
    return a * b

tool_list = [add, multiply]

In [ ]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tool_list)
llm_with_tools = llm.bind_tools(tool_list)

In [ ]:
from langgraph.graph import MessagesState

def agent(state: MessagesState) -> MessagesState:
    """
    'agent' Node
    : 주어진 상태에서 메시지를 가져와 LLM 및 도구를 사용하여 응답 메시지를 생성한다.

    Args:
        - state(MessagesState): 메시지 상태를 포함하는 state

    Returns:
        - MessagesState: 응답 메시지를 포함하는 새로운 state
    """
    
    messages = state['messages']
    
    ai_message = llm_with_tools.invoke(messages)
    
    return {'messages': [ai_message]}

In [ ]:
from langgraph.graph import END

def should_continue(state: MessagesState):
    """
    : 주어진 메시지 상태를 기반으로 에이전트가 계속 진행할지 여부를 결정한다.

    Args:
        - state(MessagesState): 메시지 상태를 포함하는 state

    Returns:
        - ['tools', END]: 도구를 사용해야 하면 `tools`, 답변할 준비가 되었다면 END 반환
    """
    
    messages = state['messages']
    last_ai_message = messages[-1]
    
    # 마지막 AI 메시지가 도구 호출을 포함하고 있는지 확인
    return 'tools' if last_ai_message.tool_calls else END

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(MessagesState)

# nodes
graph_builder.add_node('agent', agent)
graph_builder.add_node('tools', tool_node)

# edges
graph_builder.add_edge(START, 'agent')
graph_builder.add_conditional_edges(
    'agent',
    should_continue,
    ['tools', END]
)
graph_builder.add_edge('tools', 'agent')

In [ ]:
graph = graph_builder.compile()

In [ ]:
graph

In [ ]:
from langchain_core.messages import HumanMessage

for chunk in graph.stream({'messages': [HumanMessage('3에다 5를 더하고, 그 결과에 8을 곱하면?')]}, stream_mode='values'):
    chunk['messages'][-1].pretty_print()